In [41]:
# # Generalized function using selenium to get company CIN, Name and Address into a csv file.

# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
# from webdriver_manager.chrome import ChromeDriverManager
# from bs4 import BeautifulSoup
# import requests
# import pandas as pd
# import time

# def Zauba_companydetails(search):
# #     driver = webdriver.Chrome()
#     driver = webdriver.Chrome(ChromeDriverManager().install())
#     driver.get("https://www.zaubacorp.com/")

#     # Finding the search field by id
#     search_box = driver.find_element(by=By.ID, value="searchid")
#     time.sleep(3)

#     # Sending input text to search field
#     search_box.send_keys(search)

#     search_button = driver.find_element(by=By.ID, value="edit-submit--3").click()
#     time.sleep(3)

#     url = driver.current_url #get the current page url
#     driver.close() #close the browser window

#     # using BeautifulSoup to extract the table
#     page = requests.get(url)
# #     soup = BeautifulSoup(page.text, 'html.parser')

#     try:
#         df = pd.read_html(page.text)
#         data = df[0]
#         data.to_csv('Zauba_'+search+'_info.csv', index=False) # Save file as .csv with searched company name
#         print("File successfully saved!")
#     except:
#         print("No Table found. Try again")

In [45]:
# Generalized function using Beautifulsoup to get company CIN, Name and Address into a csv file.

from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

def Zauba_companydetails(search):
    url = "https://www.zaubacorp.com/companysearchresults/"+search
    
    # using BeautifulSoup to extract the table
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')

    try:
        df = pd.read_html(page.text)
        data = df[0]
        data.to_csv('Zauba_'+search+'_info.csv', index=False) # Save file as .csv with searched company name
        print("File successfully saved!")
    except:
        print("No Table found. Try again")

In [46]:
search = input("Kindly provide Company name for the details --> ")
Zauba_companydetails(search)

Kindly provide Company name for the details --> ey
File successfully saved!


In [59]:
search = 'ey'
url = "https://www.zaubacorp.com/companysearchresults/"+search

# using BeautifulSoup to extract the table
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

try:
    df = pd.read_html(page.text)
    data = pd.DataFrame(df[0])
except:
    print("No Table found. Try again")

In [63]:
data.iloc[:,0]

0                   AAK-6624
1                   AAI-5216
2      U72300UP2015PTC074294
3      U24239MH1990PTC059091
4                   AAL-2743
               ...          
495    U74999GJ2018PTC102606
496    U74999HR2018PTC074919
497    U74300RJ2001PTC017246
498    U24100TN2021PTC142773
499    U31100DL2012PTC232806
Name: CIN, Length: 500, dtype: object

### Final Zauba Function

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import string
import re
import tkinter
import customtkinter
from tkinter import *
from tkinter import messagebox
from tkinter.ttk import *
import time
from tqdm import tqdm, trange
import sys


# Generalized function using Beautifulsoup to get company CIN, Name and Address into a csv file.

# search = input("Kindly provide Company name for the details --> ")

def msg2():
    response = messagebox.showinfo('Information', 'The searched company has no result.\nKindly check.')
    customtkinter.CTkLabel(root, text=response)

def Zauba_webscrap(search):
    
    
    # URL for first search
    url = "https://www.zaubacorp.com/companysearchresults/"+search

    # using BeautifulSoup to extract the table
    page = requests.get(url)
    # soup = BeautifulSoup(page.text, 'html.parser')

    try:
        df = pd.read_html(page.text)
        data = pd.DataFrame(df[0])

    except:
#             print("No Table found. Try again")
            msg2()

    # generating the urls from the company dataframe created earlier
    df = data

    # required URl format https://www.zaubacorp.com/company-directors/'company_name_with(-)_on_whitespaces'/'CIN'


    lists=[]

    for i in range(len(df)):
        chars = re.escape(string.punctuation)
        company = df.iloc[i,1]
        company = company.replace(' & ', ' ')
        company = company.replace(' (', ' ')
        company = company.replace(") ", ' ')
        company = company.replace(' ', '-')
        list3 = company.split('-')
        company = "-".join(list3)
        company = re.sub(r'['+chars+']', '-', company)
        # company = re.sub('[^a-zA-Z0-9\n\.' ']', "-", df.iloc[3,1])
    #     print(company)

        cin = df.iloc[i,0]
    #     print(cin)
        #generating director page urls for each company
        url = 'https://www.zaubacorp.com/company-directors/'+company+'/'+cin

        cins=[]
        dic=[]
        name=[]
        designation=[]
        doa=[]

        lists.append(url)

    pbar = tqdm(total=len(lists))

    # extraction and dataframe creation for the director table
    for i in range(len(lists)):
        cin=lists[i].split('/')[-1]

        time.sleep(0.1)
        pbar.update(1)

        try:
            check=pd.read_html(lists[i])
            dfd=check[0]

        except:
            check=[]
            cins.append(cin)        
            dic.append("NA")
            name.append("NA")
            designation.append("NA")
            doa.append("NA")

        if len(check)>0:
            # dic = Director Identification Number
            for i in dfd['Director Identification Number']:
                dic.append(str(i).zfill(8))

            # name = Name
            for i in dfd['Name']:
                name.append(i)

            # designation = Designation
            for i in dfd['Designation']:
                designation.append(i)

            # doa = Date of Appointment
            for i in dfd['Date of Appointment']:
                doa.append(i)
                cins.append(cin)

    pbar.close()


    data1={
        'CIN':cins,
        'Director Identification Number':dic,
        'Director Name':name,
        'Designation':designation,
        'Date of Appointment':doa
    }

    data2=pd.DataFrame(data1)

    newdf = data.merge(data2, how='left', on='CIN')
    newdf = newdf.fillna("NA")

    # Extracting another level of Company for each Director


    lists2=[]

    for j in range(len(newdf)):
        chars = re.escape(string.punctuation)
        dir_name = re.sub(r'['+chars+']', '-', newdf.iloc[j,4]).replace(' ', '-')
    #     print(dir_name)

        din = newdf.iloc[j,3]
        din2 = str(din).zfill(8)
    #     print(din)

        #generating level 2 company page urls for each director
        url2 = 'https://www.zaubacorp.com/director/'+dir_name+'/'+din2
    #     url2 = 'https://www.zaubacorp.com/director/HIRESH-WADHWANI/02147022'

        dins=[]
        companys=[]
        designations=[]
        odoa=[]

        lists2.append(url2)


    pbar2 = tqdm(total=len(lists2))

    # extraction and dataframe creation for the level 2 company table
    for i in range(len(lists2)):
        din2 = lists2[i].split('/')[-1]

        try:
            check=pd.read_html(lists2[i])
            dfd2=check[0]

        except:
            check=[]
            dins.append(din2)        
            companys.append("NA")
            designations.append("NA")
            odoa.append("NA")

        if len(check)>0:
            # companys = Company
            for i in dfd2['Company']:
                companys.append(i)

            # designation = Designation
            for i in dfd2['Designation']:
                designations.append(i)

            # odoa = Original Date of Appointment
            for i in dfd2['Original Date of Appointment']:
                odoa.append(i)
                dins.append(din2)

        time.sleep(0.1)
        pbar2.update(1)

    pbar2.close()

    data2={
        'Director Identification Number':dins,
        'Company':companys,
        'Designation in other company':designations,
        'Original Date of Appointment':odoa
    }

    data3=pd.DataFrame(data2)
    newdf2 = newdf.merge(data3, how='left', on='Director Identification Number')
    newdf2 = newdf2.fillna("NA")
#     newdf2.to_csv('level2_company_list.csv', index = False)


    # Extracting directors for each level 2 companies
    company2 = []
    cin2=[]

    pbar3 = tqdm(total=len(newdf2))

    for i in range(len(newdf2)):
        chars = re.escape(string.punctuation)
        company = newdf2.iloc[i,7]
        company = company.replace(' & ', ' ')
        company = company.replace(' (', ' ')
        company = company.replace(") ", ' ')
        company = company.replace(' ', '-')
        list3 = company.split('-')[:3]
        company = "-".join(list3)
        company = re.sub(r'['+chars+']', '-', company)
    #     company_2 = re.sub(r'['+chars+']', '-', company.replace(' ', '-'))
    # #     company_2 = re.sub('[^a-zA-Z0-9\n\.' ']', "-", newdf2.iloc[i,7])
    #     company_2 = company[0:20]
    #     print(company_2)

        url3 = 'https://www.zaubacorp.com/companysearchresults/'+company
        if company!="NA":
            try:
                check=pd.read_html(url3)
                dfd=check[0]

            except:
                check=[]
                cin2.append("NA")
                company2.append(company)
        else:
            cin2.append("NA")
            company2.append("NA")

        if len(check)>0 and company!="NA":
            cin = dfd.iloc[0,0]
            cin2.append(cin)
            company3 = dfd.iloc[0,1]
            company2.append(company3)

        time.sleep(0.1)
        pbar3.update(1)

    pbar3.close()


    data4={
        'Company':company2,
        'CIN':cin2
    }

    df2=pd.DataFrame(data4)

    newdf2['Company'] = df2['Company']
    newdf2['CIN_2'] = df2['CIN']
    

    # Last directors extraction
    lists4=[]

    for i in range(len(df2)):
        chars = re.escape(string.punctuation)
        company = str(df2.iloc[i,0]) ##
        company = company.replace(' & ', ' ')
        company = company.replace(' (', ' ')
        company = company.replace(") ", ' ')
        company = company.replace(' ', '-')
        list3 = company.split('-')[:3]
        company = "-".join(list3)
        company = re.sub(r'['+chars+']', '-', company)

        cin = str(df2.iloc[i,1])
    #     print(cin)
        #generating director page urls for each company
        url4 = 'https://www.zaubacorp.com/company-directors/'+company+'/'+cin

        cins=[]
        dic=[]
        name=[]
        designation=[]
        doa=[]

        lists4.append(url4)

    pbar4 = tqdm(total=len(lists4))

    # extraction and dataframe creation for the director table
    for i in range(len(lists4)):
        cin=lists4[i].split('/')[-1]
        company=lists4[i].split('/')[-2] ##

        time.sleep(0.1) ##
        pbar4.update(1) ##

        try:
            check=pd.read_html(lists4[i])
            dfd=check[0]

        except:
            check=[]
            cins.append(cin)        
            dic.append("NA")
            name.append("NA")
            designation.append("NA")
            doa.append("NA")

        if len(check)>0 and company != 'nan' and cin != 'nan':
            # dic = Director Identification Number
            for i in dfd['Director Identification Number']:
                dic.append(str(i).zfill(8))

            # name = Name
            for i in dfd['Name']:
                name.append(i)

            # designation = Designation
            for i in dfd['Designation']:
                designation.append(i)

            # doa = Date of Appointment
            for i in dfd['Date of Appointment']:
                doa.append(i)
                cins.append(cin)

        else:
            cins.append(cin)        
            dic.append("NA")
            name.append("NA")
            designation.append("NA")
            doa.append("NA")



    pbar4.close()


    data5={
        'CIN_2':cins,
        'Director Identification Number_2':dic,
        'Director Name_2':name,
        'Designation_2':designation,
        'Date of Appointment_2':doa
    }

    data6=pd.DataFrame(data5)

    newdf3 = newdf2.merge(data6, how='left', on='CIN_2')
    newdf3 = newdf3.fillna("NA")
    newdf3.to_csv(search+'extracted_list.csv', index = False)
    

    dialog2.destroy()

    response = messagebox.askyesno("Message", "Thank you!\nWould you like to perform another search?")
    if response == 1:
        # return to the window
        exit()
        E1.delete(0, 'end')

    else:
        myButton['state'] = DISABLED
        myButton2['state'] = NORMAL
        # close the window
        exit()


### CustomKinter GUI

In [4]:
# pip install customtkinter --upgrade

In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time
import string
import re
import tkinter
import customtkinter
from tkinter import *
from tkinter import messagebox
from tkinter.ttk import *
import time
from tqdm import tqdm, trange
import sys


customtkinter.set_appearance_mode("System")  # Modes: system (default), light, dark
customtkinter.set_default_color_theme("blue")  # Themes: blue (default), dark-blue, green

root = customtkinter.CTk()  # create CTk window like you do with the Tk window
root.geometry("600x240")

# adding title to the frame
root.title('Multi-web Scraping Tool')

# Creating labels for the input boxes
myLabel = customtkinter.CTkLabel(root, text="Kindly select the desired website to scrap data:")
myLabel.grid(column=1, row=1)

# Dropdown menu options
options = ["Zauba", "Yahoo Finance", "LinkedIn","News website"]

optionmenu_var = customtkinter.StringVar(value="Select Website")

# Create Dropdown menu
combobox = customtkinter.CTkOptionMenu(root, values=options, variable=optionmenu_var)
combobox.grid(column=1, row=3)

L1 = customtkinter.CTkLabel(root, text="").grid(column=0, row=0)
L2 = customtkinter.CTkLabel(root, text="").grid(column=2, row=2)
L3 = customtkinter.CTkLabel(root, text="").grid(column=0, row=4)
L4 = customtkinter.CTkLabel(root, text="").grid(column=0, row=6)

# error message
def msg3():
    response = messagebox.showerror("Error", "Kindly select a website")
    customtkinter.CTkLabel(root, text=response)

# function for submit button
def button_click_event():
    
    if combobox.get()=="Select Website":
        msg3()
    
    elif combobox.get()=="Zauba":
        dialog = customtkinter.CTkToplevel()
        dialog.geometry("500x350")
        dialog.title("Zauba web scraping tool")
        
        dialog.grab_set() # when you show the popup

        
        Label1 = customtkinter.CTkLabel(dialog, text="Type in a company name:")
        Label1.grid(column=1, row=1)
        
        L1 = customtkinter.CTkLabel(dialog, text="").grid(column=0, row=0)
        L2 = customtkinter.CTkLabel(dialog, text="").grid(column=2, row=2)
        L3 = customtkinter.CTkLabel(dialog, text="").grid(column=3, row=4)
        L4 = customtkinter.CTkLabel(dialog, text="").grid(column=2, row=6)
        L5 = customtkinter.CTkLabel(dialog, text="").grid(column=2, row=8)
                
        # Creating input box
        E1 = customtkinter.CTkEntry(dialog, 
                                       width=200, 
                                       height=25,
                                       border_width=2,
                                       corner_radius=10)
        E1.configure(fg_color=('White'), text_color=('Black'))
        E1.grid(column=1, row=3)
        
        # Making the function for the submit button
        def submit_popup():
            search = E1.get()
            
            def msg1():
                response = messagebox.showerror("Error", "The tool does not accept blank entries.\nKindly enter valid name!")
                customtkinter.CTkLabel(root, text=response)
                
            if (search==""):
                msg1()
                
            else:            
                
                # make main window invisible
#                 root.withdraw()

                dialog.grab_release()
                
                # Extraction message pop up
                dialog2 = customtkinter.CTkToplevel()
                dialog2.geometry("300x50")
                dialog2.title("Extracting")

                Label_1 = customtkinter.CTkLabel(dialog2, text="Work in Progress.../.")
                Label_1.pack()
                
                #Disable the Close Window Control Icon
                def interupt_event():
                    sys.exit
                    dialog2.destroy()
                
                dialog2.protocol("WM_DELETE_WINDOW", interupt_event)
                
                dialog2.grab_set()
                                
                Zauba_webscrap(search)
            


        # Making the function for the excel open button
        def open_excel():
            # open excel file
            # method1
            
#             os.system('start EXCEL.EXE '+search+'_directors_list.csv')
            os.system('start EXCEL.EXE last_company_list_2.csv')

            # method2
            # os.system('start "excel" search+"_directors_list.csv"')

            # method3
            # file = search+"_directors_list.csv"
            # os.startfile(file)

            # destroy after opening the file
            dialog.destroy()

        # Creating button
        myButton = customtkinter.CTkButton(dialog, text="Submit", command = submit_popup)
        myButton.grid(column=1, row=5, padx=5)

        myButton2 = customtkinter.CTkButton(dialog, text="View file", command = open_excel, state = DISABLED)
        myButton2.grid(column=1, row=9, padx=5)
        
#         bar = Progressbar(ttk,orient=HORIZONTAL,length=300)
#         progressbar.configure(fg_color=..., border_color=..., progress_color=..., border_witdh=..., variable=...)
#         progressbar.grid(column=1, row=7)
        
        dialog.mainloop()
    
    elif combobox.get()=="Yahoo Finance":
        pass
    
    elif combobox.get()=="News website":
        pass
    
    
# Use CTkButton instead of tkinter Button
myButton = customtkinter.CTkButton(root, text="Submit", command=button_click_event)
myButton.grid(column=1, row=5, padx=5)

root.mainloop()
# root.destroy()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Akkshit\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\Akkshit\anaconda3\lib\site-packages\customtkinter\widgets\ctk_entry.py", line 215, in entry_focus_out
    self.set_placeholder()
  File "C:\Users\Akkshit\anaconda3\lib\site-packages\customtkinter\widgets\ctk_entry.py", line 204, in set_placeholder
    self.entry.insert(0, self.placeholder_text)
  File "C:\Users\Akkshit\anaconda3\lib\tkinter\__init__.py", line 3056, in insert
    self.tk.call(self._w, 'insert', index, string)
_tkinter.TclError: wrong # args: should be ".!ctktoplevel.!ctkentry.!entry insert index text"
  4%|███                                                                              | 19/500 [00:27<11:34,  1.44s/it]

### GUI

In [ ]:
# # Creating thourth class (OOPs)

# class App(customtkinter.CTk)
#     def __init__(self):
#         super().__init__()

#         self.geometry(f"{600}x{500}")
#         self.title("CTk example")

#         ... create widgets ...

#     ... program methods ...

# app = App()
# app.mainloop()

In [1]:
# from tkinter import *
# from tkinter import ttk
# from tkinter import messagebox
# import pandas as pd
# import numpy as np
# import csv
# import os

# # Creating an instance of tkinter frame
# root = Tk()
# # set the background colour of GUI window 
# root.configure(background='light green') 
# # adding title to the frame
# root.title('Web Scraping Tool')
# # set the configuration of GUI window
# root.geometry("275x175")
# # frm = ttk.Frame(root, padding=10)

# # Creating labels for the input boxes
# myLabel = Label(root, text="Kindly select the desired website to scrap data:", bg="light green").grid(column=1, row=1)

# # Change the label text
# def show():
#     label.config(text = clicked.get())

# # Dropdown menu options
# options = [
#     "Zauba",
#     "Yahoo FInance"
# ]
  
# # datatype of menu text
# clicked = StringVar()
  
# # initial menu text
# clicked.set( "Select website" )
  
# # Create Dropdown menu
# drop = OptionMenu(root, clicked, *options).grid(column=1, row=3)
# # drop.pack()

# L1 = Label(root, text=" ", bg="light green").grid(column=0, row=0)
# L2 = Label(root, text=" ", bg="light green").grid(column=0, row=2)
# L3 = Label(root, text=" ", bg="light green").grid(column=0, row=4)
# L4 = Label(root, text=" ", bg="light green").grid(column=0, row=6)

# # Creating button
# myButton = Button(root, text="Submit", command = show)
# myButton.grid(column=1, row=5, padx=5)

# # myButton2 = Button(root, text="View file", command = open_excel, state = DISABLED)
# # myButton2.grid(column=0, row=5, padx=5)

# root.mainloop()